We wish to minimize $<E>$ for our trial wavefunction. In Fock-Space the gradient is ${\bf{H}} \cdot \bf{c}$. We can try something like steepest descent.  

${\bf{c}}_{new} = {\bf{c}} - {\bf{H}} \cdot {\bf{c}}$

Since the current vector can be represented as a linear combination of eigenvectors of ${\bf{H}}$, the eigenvector with the largest magnitude of  eigenvalue will always dominate.  

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML, Image



basisSize = 300
lowBound  = -10.0
highBound = 10.0
xPts = np.linspace(lowBound, highBound, basisSize)


def potentFunc(x):
    #if x < -5.0:
    #    return 5.0
    #elif x > 5.0:
    #    return 5.0
    #elif x<1.0 and x> -1.0:
    #    return 2.0
    #else:
    #    return 0.0
    #return 0.02* np.abs(x)
    
    return 0.02* x**2
    

def getRepresentation(func):
    xVal = np.linspace(lowBound, highBound, basisSize)
    coeff = []
    for value in xVal:
        coeff.append(func(value))
    #coeff = func(xVal)
    return coeff
    
def makeT():
    T = np.eye(basisSize)*2.0
    for i in range(basisSize-1):
            T[i,i+1] = -1.0 
            T[i+1, i ] = -1.0
    return T

def makeV():
    vVals = getRepresentation(potentFunc)
    V = np.diag(vVals)
    return V
    
    
def getH():
    T = makeT()
    V = makeV()
    H = T+V
    return H
    
def diagH():
    H = getH()
    eigVal, eigVec = np.linalg.eigh(H)
    
    sEigVal = sorted(eigVal)
    sEigVec = np.zeros((basisSize, basisSize))
    for i, val in enumerate(sEigVal):
        j = np.where(eigVal == val)
        sEigVec[:,i]= eigVec[:, j[0][0]]
    return sEigVal, sEigVec            
    
def makePlot(state):
    eVal, eVec = diagH()
    x = np.linspace(lowBound, highBound, basisSize)
    print eVec[:, state]
    coeff = []
    for value in x:
        coeff.append(potentFunc(value))
    
    plt.plot(x, eVec[:,state]+eVal[state])
    plt.plot(x, coeff)
    plt.show()    
    
def makeVector(weightList):
    eVal, eVec = diagH()
    print eVal
    print eVec
    vector = np.zeros(basisSize)
    for i in range(len(weightList)): 
        vector += weightList[i]* eVec[:, i]
    vector = vector/np.linalg.norm(vector)
    return vector 


def solveWavefunction(vec0):
    H = getH()
    vec = vec0
    for i in range(50):
        vec += np.dot(H,vec)
        vec = vec/np.linalg.norm(vec)
        print vec
    
    return vec

def nextWave(vec0):
    H = getH()
    vec0 -= np.dot(H, vec0)
    return vec0/np.linalg.norm(vec0)


#######################################

fig, ax = plt.subplots(1,1)
Q, = ax.plot([], [])
ax.set_ylim(-0.25,0.25)
eVal, eVec = diagH()

ax.plot(xPts, eVec[:,299])

def init():
    # the init function will draw the initial frames for our animation.  In this case, we set the data in our 
    # quiver object to nothing and the text to nothing
    Q.set_data([], [])
    R = ax.plot(xPts, potentFunc(xPts))
    global vec0
    vec0 = makeVector([2.0, .25, 0.30, 0.5])
    return Q, 

def animate(i):
    # in animate we draw the objects that will change from frame to frame
    global vec0
    vec0 = nextWave(vec0)
    Q.set_data(xPts, vec0)
    # set the 
    return Q,

# create animation object
anim = animation.FuncAnimation(fig, animate, init_func=init, frames=100, interval=30, blit=True)

# this allows us to view the animation in the jupyter notebook 
rc('animation', html='html5')

# call on the animation object
anim


[0.0094540263997072767, 0.028350873731622279, 0.047225290111524404, 0.066077235385766958, 0.084906669160279571, 0.10371355079818234, 0.12249783941736986, 0.14125949388806161, 0.15999847283031529, 0.17871473461150805, 0.19740823734378132, 0.21607893888145055, 0.23472679681837744, 0.25335176848530438, 0.27195381094715176, 0.29053288100027724, 0.30908893516969366, 0.32762192970624859, 0.34613182058376191, 0.36461856349612243, 0.38308211385434221, 0.4015224267835667, 0.41993945712004155, 0.43833315940803497, 0.45670348789671289, 0.47505039653697045, 0.49337383897821219, 0.51167376856508673, 0.52995013833417137, 0.54820290101060642, 0.56643200900467705, 0.5846374144083456, 0.6028190689917281, 0.62097692419951722, 0.63911093114735129, 0.65722104061812436, 0.67530720305824032, 0.6933693685738076, 0.71140748692677502, 0.72942150753100388, 0.74741137944828151, 0.76537705138426715, 0.78331847168437585, 0.80123558832959352, 0.81912834893222775, 0.83699670073158539, 0.85484059058958239, 0.87265996

Instead, we try to use $\hat{K}|\psi> -<\psi|\hat{K}|\psi> = -\hat{V}|\psi> +<\psi|\hat{V}|\psi>$ or $\hat{K}|\psi> -<\psi|\hat{K}|\psi> + \hat{V}|\psi> -<\psi|\hat{V}|\psi> = {\bf{0}}$.
Which in fock space, is just
${\bf{H}} \cdot {\bf{c}} -({\bf{c}}^* \cdot {\bf{H}}\cdot {\bf{c}})\cdot {\bf{c}} ={\bf{0}}$

By minimizing the square modulus of the vector on the right we hope to obtain a gradient which can be used instead to hopefully converge with the nearest eigenvector of ${\bf{H}}$.

The result:

${\bf{H}}^{*} {\bf{H}} \cdot {\bf{c}} -({\bf{c}}^* \cdot {\bf{H}}\cdot {\bf{c}}){\bf{H}} \cdot {\bf{c}}$

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML, Image



basisSize = 300
lowBound  = -10.0
highBound = 10.0
xPts = np.linspace(lowBound, highBound, basisSize)
T = makeT()
V = makeV()
H = getH()

def k(vec):
    return np.dot(vec, np.dot(T,vec))

def v(vec):
    return np.dot(vec, np.dot(V,vec))

def h(vec):
    return np.dot(vec, np.dot(H, vec))
    
def makeS():
    #S = np.dot(T,T)+ np.dot(V,V)+ np.dot(T,V) + np.dot(V,T)
    S = np.dot(H,H)
    return S

S = makeS()

def newWave(vec):
    #vec += np.dot(S,vec)-(2+2*v(vec))*np.dot(T,vec)-(2+2*k(vec))*np.dot(V,vec)
    vec += np.dot(S,vec)-(h(vec))*np.dot(H,vec)
    return vec/np.linalg.norm(vec)
    
#######################################

fig, ax = plt.subplots(1,1)

Q, = ax.plot([], [])
ax.set_ylim(-0.25,0.25)

def init():
    # the init function will draw the initial frames for our animation.  In this case, we set the data in our 
    # quiver object to nothing and the text to nothing
    Q.set_data([], [])
    R = ax.plot(xPts, potentFunc(xPts))
    global vec0
    vec0 = makeVector([2.0, .25, 0.30, 0.5])
    return Q, 

def animate(i):
    # in animate we draw the objects that will change from frame to frame
    global vec0

    vec0 = newWave(vec0)
    Q.set_data(xPts, vec0)
    # set the 
    return Q,

# create animation object
anim = animation.FuncAnimation(fig, animate, init_func=init, frames=300, interval=50, blit=True)

# this allows us to view the animation in the jupyter notebook 
rc('animation', html='html5')

# call on the animation object
anim


[0.0094540263997072767, 0.028350873731622279, 0.047225290111524404, 0.066077235385766958, 0.084906669160279571, 0.10371355079818234, 0.12249783941736986, 0.14125949388806161, 0.15999847283031529, 0.17871473461150805, 0.19740823734378132, 0.21607893888145055, 0.23472679681837744, 0.25335176848530438, 0.27195381094715176, 0.29053288100027724, 0.30908893516969366, 0.32762192970624859, 0.34613182058376191, 0.36461856349612243, 0.38308211385434221, 0.4015224267835667, 0.41993945712004155, 0.43833315940803497, 0.45670348789671289, 0.47505039653697045, 0.49337383897821219, 0.51167376856508673, 0.52995013833417137, 0.54820290101060642, 0.56643200900467705, 0.5846374144083456, 0.6028190689917281, 0.62097692419951722, 0.63911093114735129, 0.65722104061812436, 0.67530720305824032, 0.6933693685738076, 0.71140748692677502, 0.72942150753100388, 0.74741137944828151, 0.76537705138426715, 0.78331847168437585, 0.80123558832959352, 0.81912834893222775, 0.83699670073158539, 0.85484059058958239, 0.87265996